# Two-Parameter Amp Model Training
Based on GuitarML's Automated-GuitarAmpModelling

In [ ]:
# Setup Environment
!git clone https://github.com/GuitarML/Automated-GuitarAmpModelling
%cd Automated-GuitarAmpModelling
!git submodule update --init --recursive
#!pip install -r requirements.txt --ignore-requires-python

In [ ]:
# GPU Check
import torch
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

# Configuration file format
The parameter values are between 0.0 and 1.0
```JSON
{
    "Number of Parameters": 2,
    "Data Sets":[
    {
	"Parameters": [ 0.0, 0.0 ],
	"TrainingClean": "./input.wav",
	"TrainingTarget": "./0_0_output.wav"
    },
    {
	"Parameters": [ 0.0, 0.25 ],
	"TrainingClean": "./input.wav",
	"TrainingTarget": "./0_0.25_output.wav"
    },
    ...
    ]
}
```


In [ ]:
# Upload and Extract Training Data
from google.colab import files
print("Please upload a ZIP file named 'input.zip' containing training configuration and wavefiles.")
uploaded = files.upload()

!unzip input.zip
!ls -R input/

In [ ]:
# Prepare wav files for training using prep_wav.py
!python prep_wav2.py model -p "./input/configuration.json" # --normalize true

In [ ]:
epochs = 200 #@param {type: "number"}

import json
import os

# Check if input directory exists
if not os.path.exists("./input"):
    print("ERROR: Input directory not found. Please make sure to upload and extract the input.zip file first.")
else:
    # Check if configuration.json exists
    config_path = "./input/configuration.json"
    if not os.path.exists(config_path):
        print(f"ERROR: Configuration file not found at {config_path}. Please check that input.zip contains a configuration.json file.")
    else:
        try:
            # Try to read and parse the JSON file
            with open(config_path, "r") as f:
                config = json.load(f)
            
            # Check if "Number of Parameters" field exists in the JSON
            if "Number of Parameters" not in config:
                print('ERROR: "Number of Parameters" field not found in configuration.json. Please check the format of your configuration file.')
            else:
                num_params = config["Number of Parameters"]
                print(f"Number of parameters set to {num_params}")
                input_size = 1 + num_params
                # Train the model using dist_model_recnet.py
                !python dist_model_recnet.py -l RNN3-model -eps {epochs} --seed 39 -is {input_size}
        except json.JSONDecodeError:
            print(f"ERROR: Failed to parse {config_path}. The file is not valid JSON. Please check the format of your configuration file.")
        except Exception as e:
            print(f"ERROR: An unexpected error occurred while reading the configuration file: {str(e)}")


In [ ]:
# Name your config in the following command
# The plots will be generated in the Results/modelName/ directory
!python plot.py model